In [1]:
import torch
import numpy as np

import cv2

In [3]:
x = torch.rand(10, 4)

In [15]:
torch.linalg.norm(x, ord=2, dim=1, keepdim=True)

tensor([[1.2319],
        [1.4829],
        [1.3308],
        [0.9505],
        [1.3126],
        [1.4152],
        [1.1665],
        [1.2365],
        [0.6988],
        [0.8920]])

In [16]:
x.pow(2).sum(1, keepdim=True).sqrt()

tensor([[1.2319],
        [1.4829],
        [1.3308],
        [0.9505],
        [1.3126],
        [1.4152],
        [1.1665],
        [1.2365],
        [0.6988],
        [0.8920]])

In [8]:
x / x.sum(1, keepdim=True).sqrt()

tensor([[0.3953, 0.2838, 0.3821, 0.4943],
        [0.5452, 0.3409, 0.3397, 0.4781],
        [0.5194, 0.6322, 0.0179, 0.3352],
        [0.4220, 0.4440, 0.0718, 0.3771],
        [0.4666, 0.3174, 0.5570, 0.2423],
        [0.4578, 0.1988, 0.4204, 0.5640],
        [0.7281, 0.0205, 0.4669, 0.1200],
        [0.0891, 0.1728, 0.6864, 0.4850],
        [0.4679, 0.0517, 0.2389, 0.3473],
        [0.3398, 0.3482, 0.4792, 0.1196]])

In [85]:
def normalize_points(grid: torch.Tensor, K: torch.Tensor, scales: tuple=(1., 1.)):
    ''' 
    :param grid: coordinates (u, v), B x 2 x H x W
    :param K: intrinsics, B x 3 x 3
    :param scales: parameters of resizing of original image
    '''    
    fx, fy, ox, oy = K[:, 0, 0], K[:, 1, 1], K[:, 0, 2], K[:, 1, 2]
    sx, sy = scales[0], scales[1]
    fx *= sx
    fy *= sy
    ox *= sx
    oy *= sy
    principal_point = torch.cat([ox[..., None], oy[..., None]], -1)[..., None, None]
    focal_length = torch.cat([fx[..., None], fy[..., None]], -1)[..., None, None]
    return (grid - principal_point) / focal_length

def create_normalized_grid(size: tuple, K: torch.Tensor, scales: tuple=(1., 1.)):
    '''Given image size, return grid for pixels positions, normalized with intrinsics (K).
    :param size: image size (B, H, W)
    :param K: intrinsics, B x 3 x 3
    :param scales: parameters of resizing of original image
    '''
    B, H, W = size
    grid = torch.meshgrid((torch.arange(H), torch.arange(W)), indexing='ij')
    grid = torch.cat((grid[1].unsqueeze(0), grid[0].unsqueeze(0)), dim=0).float()
    grid = grid[None, ...].repeat(B, 1, 1, 1)
    grid = normalize_points(grid, K, scales)
    return grid

In [86]:
grid = create_normalized_grid((2, 120, 160), K_, (1., 1.))

In [87]:
grid

tensor([[[[-0.5529, -0.5512, -0.5494,  ..., -0.2812, -0.2795, -0.2777],
          [-0.5529, -0.5512, -0.5494,  ..., -0.2812, -0.2795, -0.2777],
          [-0.5529, -0.5512, -0.5494,  ..., -0.2812, -0.2795, -0.2777],
          ...,
          [-0.5529, -0.5512, -0.5494,  ..., -0.2812, -0.2795, -0.2777],
          [-0.5529, -0.5512, -0.5494,  ..., -0.2812, -0.2795, -0.2777],
          [-0.5529, -0.5512, -0.5494,  ..., -0.2812, -0.2795, -0.2777]],

         [[-0.4145, -0.4145, -0.4145,  ..., -0.4145, -0.4145, -0.4145],
          [-0.4127, -0.4127, -0.4127,  ..., -0.4127, -0.4127, -0.4127],
          [-0.4110, -0.4110, -0.4110,  ..., -0.4110, -0.4110, -0.4110],
          ...,
          [-0.2120, -0.2120, -0.2120,  ..., -0.2120, -0.2120, -0.2120],
          [-0.2103, -0.2103, -0.2103,  ..., -0.2103, -0.2103, -0.2103],
          [-0.2085, -0.2085, -0.2085,  ..., -0.2085, -0.2085, -0.2085]]],


        [[[-0.5529, -0.5512, -0.5494,  ..., -0.2812, -0.2795, -0.2777],
          [-0.5529, -0.5512,

In [69]:
K_ = torch.from_numpy(K).unsqueeze(0).repeat(2, 1, 1)

In [82]:
out = normalize_points(grid, K_)

In [84]:
create_normalized_grid()

tensor([[[-0.5529, -0.5512, -0.5494,  ..., -0.2812, -0.2795, -0.2777],
         [-0.5529, -0.5512, -0.5494,  ..., -0.2812, -0.2795, -0.2777],
         [-0.5529, -0.5512, -0.5494,  ..., -0.2812, -0.2795, -0.2777],
         ...,
         [-0.5529, -0.5512, -0.5494,  ..., -0.2812, -0.2795, -0.2777],
         [-0.5529, -0.5512, -0.5494,  ..., -0.2812, -0.2795, -0.2777],
         [-0.5529, -0.5512, -0.5494,  ..., -0.2812, -0.2795, -0.2777]],

        [[-0.4145, -0.4145, -0.4145,  ..., -0.4145, -0.4145, -0.4145],
         [-0.4127, -0.4127, -0.4127,  ..., -0.4127, -0.4127, -0.4127],
         [-0.4110, -0.4110, -0.4110,  ..., -0.4110, -0.4110, -0.4110],
         ...,
         [-0.2120, -0.2120, -0.2120,  ..., -0.2120, -0.2120, -0.2120],
         [-0.2103, -0.2103, -0.2103,  ..., -0.2103, -0.2103, -0.2103],
         [-0.2085, -0.2085, -0.2085,  ..., -0.2085, -0.2085, -0.2085]]])

In [28]:
K = Ks['scene0609_02'].reshape(3, 3)

In [46]:
(0. - 319.5) / 577.870
    

-0.5528925190786855

In [42]:
grid

tensor([[[[  0.,   1.,   2.,  ..., 157., 158., 159.],
          [  0.,   1.,   2.,  ..., 157., 158., 159.],
          [  0.,   1.,   2.,  ..., 157., 158., 159.],
          ...,
          [  0.,   1.,   2.,  ..., 157., 158., 159.],
          [  0.,   1.,   2.,  ..., 157., 158., 159.],
          [  0.,   1.,   2.,  ..., 157., 158., 159.]],

         [[  0.,   0.,   0.,  ...,   0.,   0.,   0.],
          [  1.,   1.,   1.,  ...,   1.,   1.,   1.],
          [  2.,   2.,   2.,  ...,   2.,   2.,   2.],
          ...,
          [117., 117., 117.,  ..., 117., 117., 117.],
          [118., 118., 118.,  ..., 118., 118., 118.],
          [119., 119., 119.,  ..., 119., 119., 119.]]]])

In [41]:
apply_inverse_intrinsics(grid, K, scales=(1., 1.))

tensor([[[[-1.0000, -0.9983, -0.9965,  ..., -0.7283, -0.7266, -0.7249],
          [-1.0000, -0.9983, -0.9965,  ..., -0.7283, -0.7266, -0.7249],
          [-1.0000, -0.9983, -0.9965,  ..., -0.7283, -0.7266, -0.7249],
          ...,
          [-1.0000, -0.9983, -0.9965,  ..., -0.7283, -0.7266, -0.7249],
          [-1.0000, -0.9983, -0.9965,  ..., -0.7283, -0.7266, -0.7249],
          [-1.0000, -0.9983, -0.9965,  ..., -0.7283, -0.7266, -0.7249]],

         [[-0.7496, -0.7496, -0.7496,  ..., -0.7496, -0.7496, -0.7496],
          [-0.7465, -0.7465, -0.7465,  ..., -0.7465, -0.7465, -0.7465],
          [-0.7433, -0.7433, -0.7433,  ..., -0.7433, -0.7433, -0.7433],
          ...,
          [-0.3834, -0.3834, -0.3834,  ..., -0.3834, -0.3834, -0.3834],
          [-0.3803, -0.3803, -0.3803,  ..., -0.3803, -0.3803, -0.3803],
          [-0.3772, -0.3772, -0.3772,  ..., -0.3772, -0.3772, -0.3772]]]])

In [31]:
np.array([[1/K[0, 0], 0., -K[0, 2]/K[0, 0]]])

array([[ 0.00173049,  0.        , -0.55289191]])

tensor([[[[-100.,  -99.,  -98.,  ...,   57.,   58.,   59.],
          [-100.,  -99.,  -98.,  ...,   57.,   58.,   59.],
          [-100.,  -99.,  -98.,  ...,   57.,   58.,   59.],
          ...,
          [-100.,  -99.,  -98.,  ...,   57.,   58.,   59.],
          [-100.,  -99.,  -98.,  ...,   57.,   58.,   59.],
          [-100.,  -99.,  -98.,  ...,   57.,   58.,   59.]],

         [[-200., -200., -200.,  ..., -200., -200., -200.],
          [-199., -199., -199.,  ..., -199., -199., -199.],
          [-198., -198., -198.,  ..., -198., -198., -198.],
          ...,
          [ -83.,  -83.,  -83.,  ...,  -83.,  -83.,  -83.],
          [ -82.,  -82.,  -82.,  ...,  -82.,  -82.,  -82.],
          [ -81.,  -81.,  -81.,  ...,  -81.,  -81.,  -81.]]]])

In [26]:
np.loadtxt('/home/project/data/ScanNet/scans/scene0609_02/intrinsic/intrinsic_depth.txt')

array([[577.870605,   0.      , 319.5     ,   0.      ],
       [  0.      , 577.870605, 239.5     ,   0.      ],
       [  0.      ,   0.      ,   1.      ,   0.      ],
       [  0.      ,   0.      ,   0.      ,   1.      ]])

In [19]:
(np.linalg.inv(Ks['scene0609_02'].reshape(3, 3)) @ np.array([640, 480, 1]).reshape(-1, 1)).T

array([[0.55462244, 0.41618312, 1.        ]])

In [23]:
cv2.imread('/home/project/data_sample/scannet_test_1500/scene0707_00/color/105.jpg').shape

(968, 1296, 3)